ADVANCE PYTHON PROJECT ASSIGNMENT

In [ ]:
# 1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
# You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud platform
pip install flask requests beautifulsoup4
from flask import Flask, render_template, request
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

def scrape_quotes():
    url = "http://quotes.toscrape.com/"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    quotes = [q.text for q in soup.select('.quote .text')]
    return quotes

@app.route('/')
def home():
    quotes = scrape_quotes()
    return render_template('scraped.html', quotes=quotes)

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 2. Create a Flask app that consumes data from external APIs and displays it to users.
# Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform
from flask import Flask, render_template
import requests

app = Flask(__name__)

@app.route('/')
def api_example():
    resp = requests.get("https://randomuser.me/api/?results=5")
    data = resp.json()['results']
    return render_template('api.html', users=data)

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.
pip install flask-authlib
from flask import Flask, url_for, redirect, session
from authlib.integrations.flask_client import OAuth

app = Flask(__name__)
app.secret_key = 'secret'
oauth = OAuth(app)

google = oauth.register(
    name='google',
    client_id='YOUR_GOOGLE_CLIENT_ID',
    client_secret='YOUR_GOOGLE_CLIENT_SECRET',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    access_token_params=None,
    authorize_url='https://accounts.google.com/o/oauth2/auth',
    authorize_params=None,
    api_base_url='https://www.googleapis.com/oauth2/v1/',
    userinfo_endpoint='https://www.googleapis.com/oauth2/v3/userinfo',
    client_kwargs={'scope': 'openid email profile'},
)

@app.route('/')
def home():
    user = dict(session).get('profile', None)
    return f"Hello, {user['name']}" if user else '<a href="/login">Login with Google</a>'

@app.route('/login')
def login():
    redirect_uri = url_for('authorize', _external=True)
    return google.authorize_redirect(redirect_uri)

@app.route('/login/callback')
def authorize():
    token = google.authorize_access_token()
    resp = google.get('userinfo')
    user_info = resp.json()
    session['profile'] = user_info
    return redirect('/')

@app.route('/logout')
def logout():
    session.pop('profile', None)
    return redirect('/')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 4. Develop a recommendation system using Flask that suggests content to users based on their preferences
from flask import Flask, render_template, session, redirect, request, url_for

app = Flask(__name__)
app.secret_key = 'secret'

# Sample content database
ITEMS = {
    "sports": ["Football News", "Basketball Analysis"],
    "music": ["Top Songs", "Album Reviews"],
    "movies": ["Latest Releases", "Oscar Winners"]
}

@app.route('/', methods=['GET', 'POST'])
def set_prefs():
    if request.method == 'POST':
        prefs = request.form.getlist('prefs')
        session['prefs'] = prefs
        return redirect(url_for('recommend'))
    return render_template('prefs.html', categories=ITEMS.keys())

@app.route('/recommend')
def recommend():
    prefs = session.get('prefs', [])
    suggestions = []
    for p in prefs:
        suggestions += ITEMS.get(p, [])
    return render_template('recommend.html', suggestions=suggestions)

if __name__ == '__main__':
    app.run(debug=True)
